# Automatic emittance measurement at LCLS
In this case we have 4 quadrupoles, 3 that are used to minimize the incoming beam and
 the fourth to perform the quad scan measurement.

In [9]:
# optionally add scripts location to path
if True:
    import sys
    sys.path.append("../../../")

import xopt
print(xopt.__version__)

# set up data saving locations
data_dir = "/home/physics3/ml_tuning/20230729_LCLS_Injector"


In [ ]:
from scripts.evaluate_function.screen_image import measure_beamsize, measure_background
from scripts.optimize_function import optimize_function
from scripts.characterize_emittance import characterize_emittance

In [ ]:
## import variable ranges
import pandas as pd
filename = "../../variables.csv"
VARIABLE_RANGES = pd.read_csv(filename, index_col=0, header=None).T.to_dict(orient='list')
SCREEN_NAME = "OTRS:IN20:621" # OTR 3

## (Optional) Measure background image

In [ ]:
MEASURE_BACKGROUND = True
BACKGROUND_FILE = f"{data_dir}/{SCREEN_NAME}_background.npy".replace(":","_")

if MEASURE_BACKGROUND:
    measure_background(SCREEN_NAME)
else:
    BACKGROUND_FILE = None

In [ ]:
# verify background image
import numpy as np
import matplotlib.pyplot as plt
plt.imshow(np.load(BACKGROUND_FILE))

## Test screen measurement

In [ ]:
res = measure_beamsize({"screen":SCREEN_NAME, "visualize": True})

## Define measurement parameters

In [ ]:
from xopt import VOCS
import numpy as np
import json
from epics import caget_many


TUNING_VARIABLES = ["QUAD:IN20:425:BCTRL","QUAD:IN20:441:BCTRL","QUAD:IN20:511:BCTRL"]
SCAN_VARIABLE = "QUAD:IN20:525"
QUAD_LENGTH = 0.108 # m
DRIFT_LENGTH = 4.17 # m
BEAM_ENERGY = 0.135 # GeV
PV_TO_INTEGRATED_GRADIENT = 1.0 # kG
ROI = None
THRESHOLD = 3000

with open("../../secondary_variables.json", "r") as f:
    secondary_observables = json.loads(f)

measurement_options = {
    "screen": SCREEN_NAME,
    "background": BACKGROUND_FILE,
    "threshold": THRESHOLD,
    "roi": ROI,
    "bb_half_width": 3.0, # half width of the bounding box in terms of std
    "visualize": True,
    "save_img_location": data_dir
}

image_constraints = {
    "bb_penalty": ["LESS_THAN", 0.0],
    "log10_total_intensity": ["GREATER_THAN", 4]
}

# define function to measure the total size on OTR4
def eval_beamsize(input_dict):
    results = measure_beamsize(input_dict)

    # get secondary PV settings/readbacks
    secondary_results = caget_many(secondary_observables)
    results = results | secondary_results


    results["S_x_mm"] = results["Sx"] * 1e3
    results["S_y_mm"] = results["Sy"] * 1e3

    #add total beam size
    results["total_size"] = np.sqrt(results["Sx"]**2 + results["Sy"]**2)
    return results



## Optimize upstream beam parameters to minimze beamsize
Note that the scan variable is fixed at zero for maximum range quadrupole scan.

In [ ]:
opt_vocs = VOCS(
        variables = {ele: VARIABLE_RANGES[ele] for ele in TUNING_VARIABLES},
        constants = {SCAN_VARIABLE: 0.0} | measurement_options,
        constraints = image_constraints,
        objectives = {"total_size": "MINIMIZE"}
    )

opt_x = optimize_function(
    opt_vocs, eval_beamsize, n_iterations=1,
)

In [ ]:
opt_x.data

## Automatic quad scan and emittance characterization

In [ ]:
from emitopt.utils import get_quad_strength_conversion_factor
emit_vocs = VOCS(
        variables = {SCAN_VARIABLE: VARIABLE_RANGES[SCAN_VARIABLE]},
        observables = ["S_x_mm", "S_y_mm"],
        constraints = image_constraints,
        constants = opt_x.data.iloc[-1][TUNING_VARIABLES].to_dict() | measurement_options
    )

# create conversion factor from PV value to geometric focusing strength
integrated_gradient_to_geometric_focusing_strength = get_quad_strength_conversion_factor(
        BEAM_ENERGY, QUAD_LENGTH
    )
quad_strength_scale_factor =  PV_TO_INTEGRATED_GRADIENT * integrated_gradient_to_geometric_focusing_strength

emit_results, emit_Xopt = characterize_emittance(
    emit_vocs,
    eval_beamsize,
    QUAD_LENGTH,
    DRIFT_LENGTH,
    quad_strength_key=SCAN_VARIABLE,
    quad_strength_scale_factor=quad_strength_scale_factor,
    rms_x_key="S_x_mm",
    rms_y_key="S_y_mm",
    quad_scan_analysis_kwargs={"visualize": True}
)

In [ ]:
emit_results

In [ ]:
emit_Xopt.data